In [1]:
import torch
import cv2
from torchvision.transforms import Compose, Resize, ToTensor, Normalize

In [ ]:
# Load MiDaS v3 (DPT-Large)
model = torch.hub.load("intel-isl/MiDaS", "DPT_Large", pretrained=True)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Preprocess RGB image
transform = Compose([Resize((384, 384)), ToTensor(), Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])
img = cv2.imread("crop_weed_image.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
input_tensor = transform(img).unsqueeze(0).to(device)

# Generate depth map
with torch.no_grad():
    depth_map = model(input_tensor)  # Shape: [1, 1, H, W]

# Save depth map
depth_map_np = depth_map.squeeze().cpu().numpy()  # Shape: [H, W]
np.save("depth_map.npy", depth_map_np)  # Save for later use

# Prepare for custom encoder (example)
depth_map_tensor = torch.from_numpy(depth_map_np).unsqueeze(0).unsqueeze(0)  # Shape: [1, 1, H, W]
# Feed depth_map_tensor (or combined with RGB) into your custom encoder